### **4.1 - Add data at runtime with templating**

Airflow tiene una característica muy poderosa, digamos que quieres obtener un segmento de datos (chunk of data) basado en la fecha y, obviamente, no quieres tener esta fecha codificada en tu tarea porque de lo contrario siempre obtendrás el mismo segmento de datos (chunk of data), así que la pregunta es cómo puedes asegurarte de que esta fecha cambiará de acuerdo a cuando se ejecute la tarea. Bueno, aquí es donde tienes que aprovechar el "template Jinja engine" y esto es extremadamente poderoso. Volviendo a tu data pipeline, para insertar datos en tiempo de ejecución (at run time), necesitas usar una notación especial y esa notación es 2 pares de llaves. Al poner 2 pares de llaves dentro del string, eso significa que hay algo aquí en tiempo de ejecución (at runtime) y eso es exactamente lo que estás haciendo ahí mismo con la variable. Al poner esos pares de llaves, este "var.json.my_dag_partner.name" será reemplazado en tiempo de ejecución cuando tu DAG se esté ejecutando por el valor correspondiente de name en la variable "my_dag_partner". Una cosa que hay que tener en cuenta es que no puedes usar "templating" donde quieras, tienes que asegurarte de que el argumento está "templado (templated)" y ¿cómo puedes saberlo? Tienes que echar un vistazo a la documentación. Así que digamos que quieres usar el operador postgres, para ilustrar esto, en ese caso tienes que importarlo primero, así que escribe "from airflow.providers.postgres.operators.postgres import PostgresOperator". 

<center><img src="https://i.postimg.cc/Xq58rLLR/a1138.png"></center>

Una vez que tienes el PostgresOperator quieres usarlo ¿verdad? así que vamos a crear una nueva tarea. Y ahora tienes el argumento "sql" así que aquí tienes que poner tu SQL request, ¿ese argumento es templado (templated) o no?

<center><img src="https://i.postimg.cc/Z5HLvCP5/a1139.png"></center>

Bueno, echemos un vistazo al registro para eso, así que si vas al "registry.astronomer.io" que es el mejor lugar para ir si quieres encontrar información sobre tu operador, vamos a escribir "postgres" y luego haz clic en "PostgresOperator" y si echas un vistazo cuidadosamente a los parámetros, tienes "sql" que se requiere, y recibe ya sea un string que representa tu sentencia SQL o una lista de sentencias SQL o incluso una referencia a un template file.

<center><img src="https://i.postimg.cc/vHRhJRHb/a1140.png"></center>

Y el punto más importante aquí es que es "templado (templated)", lo que significa que eres capaz de inyectar datos en tiempo de ejecución usando ese argumento y así eres capaz de aprovechar el template engine. Así que digamos que quieres obtener datos de una tabla SQL, en ese caso podrías escribir algo como esto:

<center><img src="https://i.postimg.cc/vmSLXkNR/a1141.png"></center>

Pero obviamente si haces eso hay un gran problema aquí, porque siempre obtendrás el mismo fragmento de datos (chunk of data) y aquí es donde debido a que el argumento SQL es templado (templated), eres capaz de usar el template engine. Y así en vez de poner esta fecha quieres usar la current execution date de tu DAG Run correspondiente al periodo de tiempo para el que se dispara tu DAG. Así que aquí puedes poner dos pares de llaves, para decir ¡eh!, esto es algo que quiero reemplazar en tiempo de ejecución y aquí pones "**`ds`**".

<center><img src="https://i.postimg.cc/pTwQM8vq/a1142.png"></center>

Y "**`ds`**" es un parámetro predefinido que corresponde a la current execution date de tu DAG Run. Puedes echar un vistazo al valor que se renderizará en cuanto se dispare tu DAG simplemente haciendo clic en tu DAG "my_dag", luego ve a "Graph View" y haz clic en la tarea "fetching_data" luego haz clic en "Rendered" y podrás ver que, si disparas tu DAG, ahora mismo obtendrás este valor, esta fecha en lugar de la "ds".

<center><img src="https://i.postimg.cc/jdzcK8LN/a1143.png"></center>

Así es como funciona, así es como funciona el templating y por eso es realmente poderoso porque puedes inyectar datos en tiempo de ejecución de acuerdo a algún contexto. Ahora puedes hacer mucho más que eso, de hecho, **`como una mejor práctica no deberías poner tu sentencia SQL directamente en tu PostgresOperator, en su lugar deberías tener un archivo SQL y poner tu SQL request en él, para que mantengas separado tu DAG de tus sentencias SQL, y eso hará tu DAG mucho más limpio`**. Así que vamos a hacer esto, si usted crea una nueva carpeta en la carpeta "dags", por ejemplo, la carpeta "sql", a continuación, crear un nuevo archivo en él, llamado "MY_REQUESTS.sql":

<center><img src="https://i.postimg.cc/nLLG4pTY/a1144.png"></center>

Entonces usted puede copiar esta SQL request en el archivo, y guarde el archivo: 

<center><img src="https://i.postimg.cc/G3Tzg3RX/a1145.png"></center>

y aquí tienes que poner la ruta de acceso al archivo que es:

<center><img src="https://i.postimg.cc/76G1R9F5/a1146.png"></center>

Y lo bueno es que eres capaz de utilizar el template engine dentro del archivo SQL como se muestra allí mismo:

<center><img src="https://i.postimg.cc/rwMJpSrH/a1147.png"></center>

Esto funciona, ¿por qué funciona? Bueno, de nuevo tienes que echar un vistazo a la documentación de tu operador. En realidad, incluso el código, por lo que si echas un vistazo a él:

<center><img src="https://i.postimg.cc/zvTCzBX5/a1148.png"></center>
<center><img src="https://i.postimg.cc/66ZLLHVS/a1149.png"></center>

puedes ver aquí "**`template_ext`**" con '**`.sql`**' que corresponde a los archivos que el Postgres operator reconocerá como un templated file y así, en esos archivos con la siguiente extensión '.sql' podrás usar el template engine y así es exactamente cómo funciona detrás de escena. Lo mismo para los argumentos puedes ver en "**`template_fields`**" que el argumento del PostgresOperator '**`sql`**' es templated.

<center><img src="https://i.postimg.cc/9FjPxGJf/a1150.png"></center>

Esos dos parámetros "template_fields" y "template_ext" son extremadamente útiles si quieres personalizar tu operador y hacer que algunos argumentos sean templated. Eso es lo que vas a ver ahora. Por defecto el argumento parameters no está templado, eso es lo que puedes ver ahí mismo:

<center><img src="https://i.postimg.cc/4Nb6jmSQ/a1151.png"></center>

Así que usted puede poner algunos parameters, pero no es capaz de inyectar los parameters en tiempo de ejecución. Así que digamos que quieres "templar (template)" ese argumento, ¿cómo puedes hacerlo? Extremadamente fácil. Copiar este "template_fields" donde, de nuevo, eso corresponde a los argumentos que se templan en sus operadores, así que copia esto, la línea entera:

<center><img src="https://i.postimg.cc/Dzm1W5yx/a1152.png"></center>

luego vuelve a tu editor de código y aquí vas a crear una custom operator class, así que vamos a definir una class CustomPostgresOperator y este operador personalizado hereda del Postgres operator y lo único que tienes que hacer es redefinir "template_fields" pero esta vez vas a añadir parámetros:

<center><img src="https://i.postimg.cc/tJG6Mjdc/a1153.png"></center>

Y eso es todo. Si utilizas el CustomPostgresOperator en lugar del PostgresOperator

<center><img src="https://i.postimg.cc/GtPD05qm/a1154.png"></center>

esta vez, los "parameters" son templados y puedes inyectar datos en tiempo de ejecución. Solo para mostrarte que funciona, definamos algunos parámetros aquí, digamos que quieres obtener la próxima fecha de ejecución (next_ds), luego quieres tal vez la fecha de ejecución anterior (prev_ds) y por qué no obtener la variable, así que pongamos "partner_name" y guardemos el archivo:

<center><img src="https://i.postimg.cc/d0XrGdhw/a1155.png"></center>

luego en la Airflow UI, ve a "my_dag", haz clic en Graph View y como puedes ver, esta vez, utiliza el CustomProgressOperator para obtener los datos. Haz clic en la tarea "fetching_data", haz clic en "Rendered" y esta vez obtienes los siguientes valores para tus parámetros :

<center><img src="https://i.postimg.cc/W15MbxjD/a1156.png"></center>
<center><img src="https://i.postimg.cc/g2zVpG9F/a1157.png"></center>
<center><img src="https://i.postimg.cc/zBgW7BhW/a1158.png"></center>
<center><img src="https://i.postimg.cc/c1fw8v0G/a1159.png"></center>

y como puedes ver obtienes la "next execution date", el "the partner name" definido en tu variable y la "previous execution date". Una última cosa para que veas que por defecto no funciona, si vuelves a tu data pipeline y cambias CustomPostgresOperator por PostgresOperator guarda el archivo:

<center><img src="https://i.postimg.cc/sggh8KS3/a1160.png"></center>

y luego vuelves a tus DAGs, luego haces click en "my_dag", el click en la Graph view, haces click en la tarea "fetching_data", haces click en "Rendered" y como puedes ver que esta vez no obtienes ningún valor ya que los parámetros no están templados.

<center><img src="https://i.postimg.cc/WbnZXxd9/a1161.png"></center>